# this notebook shows backtestlib.optimizer.BenchMark etc.

In [1]:
import sys
sys.path.insert(0, "..")

import logging as log
log.basicConfig(level = log.INFO, stream=sys.stdout,\
                format='backtest-%(asctime)s-%(funcName)s: %(message)s', \
                datefmt='%Y-%b-%d %H:%M:%S')
from  backtestlib.optimizer import BenchMark, SampleBased, SpectralCut, SpectralSelection,ShrinkToIdentity,POET
import pandas as pd
import numpy as np
import numpy.linalg as LA
%load_ext autoreload
%autoreload 2

In [2]:
signal = pd.Series(data = [1,2,2,1.5,-.2],index = ['AAPL','GOOGLE','MMM','BAC','GS'])
signal

AAPL      1.0
GOOGLE    2.0
MMM       2.0
BAC       1.5
GS       -0.2
dtype: float64

In [3]:
c = 0.02
r = 0.00
n = 50
p = len(signal)

rand = np.random.RandomState(1233)
true_var = np.diag(np.ones(p))*(1-c) + c*np.ones((p,p))
true_mu = np.ones(p)*r

returns = rand.multivariate_normal(true_mu, true_var, n).T
sample_var =(returns@returns.T)/n

In [4]:
opt = BenchMark()
opt.optimize(signal = signal)

backtest-2021-Feb-22 01:54:59-__init__: BenchMark initialized


array([0.2, 0.2, 0.2, 0.2, 0.2])

In [5]:
opt = SampleBased()
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SampleBased initialized


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

In [6]:
opt = SpectralCut(delta = 0.4)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SpectralCut initialized with  delta 0.4
backtest-2021-Feb-22 01:54:59-optimize: Optimal K is 5
backtest-2021-Feb-22 01:54:59-optimize:  return component [-0.56292848  0.56166544 -1.79661716 -0.34753213  2.70352208]


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

In [7]:
opt = ShrinkToIdentity()
opt.optimize(signal = signal,returns = returns)

backtest-2021-Feb-22 01:54:59-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-22 01:54:59-optimize: shrink coefficients: a 1.708, b 0.3, d 1.734, m 0.858


array([ 0.1141222 ,  0.40128602,  0.42972691,  0.16964652, -0.11478166])

In [8]:
opt = SpectralSelection(delta = 0.4, c = 10)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SpectralCut initialized with  delta 0.4, c 10
backtest-2021-Feb-22 01:54:59-optimize: Optimal gamma is 0.003
backtest-2021-Feb-22 01:54:59-optimize:  return component [-0.56266804  0.55608824 -1.7810237  -0.2796166   1.36131824]


array([ 0.16811186,  0.28152213,  0.30907235,  0.28438214, -0.04308848])

- To demostrate POET we simulate returns via a factor model of K factors such that K << p~n
- K: number of factors, p asset numbers, n time periods

In [9]:
c = 0.3
n = 200
p = 100

rand = np.random.RandomState(1234)
temp = rand.normal(size = (p,n))
temp_u,temp_s,temp_vh =  LA.svd(temp)

true_K = 6
true_B = temp_u[:,:true_K]@np.diag(temp_s[:true_K])
true_F = temp_vh.T[:,:true_K]
errors = rand.normal(size = (p,n))*c
returns = true_B@true_F.T + errors

signal = pd.Series(np.ones(p))

In [10]:
opt = POET(C = 0.02)
opt.optimize(signal = signal,returns = returns)

backtest-2021-Feb-22 01:54:59-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-22 01:54:59-optimize: Optimal K is 6
backtest-2021-Feb-22 01:54:59-optimize: offdiagnal f-norm under C=0.02 is 0.577


array([ 0.01157585, -0.00949097,  0.00741841,  0.014982  ,  0.00874337,
        0.01065229,  0.02104509,  0.00414328,  0.02327296,  0.00185187,
       -0.00211298,  0.00859463,  0.0073677 ,  0.01502131,  0.00471561,
        0.00397781,  0.01855608,  0.00093859,  0.01727918,  0.01309124,
        0.01271723,  0.01882545,  0.01830256, -0.00273349,  0.00980433,
        0.0288522 ,  0.01564092,  0.02637702, -0.00311143,  0.00986773,
        0.00229348,  0.00552057,  0.0039822 ,  0.01607647,  0.00918691,
        0.00027285,  0.02131201,  0.02195193,  0.02102599,  0.0155175 ,
        0.01333615,  0.01571371,  0.01467245,  0.00398114,  0.01566309,
        0.01355034,  0.01217234,  0.01744387, -0.00244282, -0.0076788 ,
        0.00111382,  0.00705849,  0.00491745,  0.0070263 ,  0.01941909,
        0.01787315,  0.01115552,  0.00205773,  0.00539261,  0.00115094,
       -0.00857944,  0.00568472,  0.0094757 , -0.00090542,  0.00525649,
        0.00336744,  0.00689343,  0.01238462,  0.0101022 ,  0.01